<a href="https://colab.research.google.com/github/carol-furtado/Denoising-and-image-classification/blob/main/Image_Denoising_%26_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [4]:
from google.colab import files
uploaded = files.upload()


Saving train.zip to train.zip


In [5]:
import zipfile

with zipfile.ZipFile("train.zip", 'r') as zip_ref:
    zip_ref.extractall("train")


In [8]:
!ls /content


sample_data  train  train.zip


In [12]:
data_dir_clean = "/content/train/train/clean"
data_dir_noisy = "/content/train/train/noisy"
normalization_layer = tf.keras.layers.Rescaling(1./255)

clean = tf.keras.utils.image_dataset_from_directory(
    data_dir_clean,
    labels="inferred",          # get class from folder name
    label_mode="int",           # integer labels (0,1,2,...)
    image_size=(224,224),
    batch_size=32,
    shuffle=False
).map(lambda x, y: (normalization_layer(x), y))


noisy = tf.keras.utils.image_dataset_from_directory(
    data_dir_noisy,
    labels="inferred",          # must match clean folder structure
    label_mode="int",
    image_size=(224,224),
    batch_size=32,
    shuffle=False
).map(lambda x, y: (normalization_layer(x), y))


Found 1795 files belonging to 5 classes.
Found 1795 files belonging to 5 classes.


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class ConvAutoencoder(Model):
    def __init__(self, latent_dim=128):
        super(ConvAutoencoder, self).__init__()

        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(224, 224, 3)),
            layers.Conv2D(32, (3,3), activation='relu', padding='same', strides=2),  # 112x112x32
            layers.Conv2D(64, (3,3), activation='relu', padding='same', strides=2),  # 56x56x64
            layers.Conv2D(128, (3,3), activation='relu', padding='same', strides=2), # 28x28x128
            layers.Conv2D(256, (3,3), activation='relu', padding='same', strides=2), # 14x14x256
            layers.Flatten(),
            layers.Dense(latent_dim, activation='relu')  # latent representation
        ])

        # Decoder
        self.decoder = tf.keras.Sequential([
            layers.Dense(14*14*256, activation='relu'),
            layers.Reshape((14, 14, 256)),
            layers.Conv2DTranspose(128, (3,3), strides=2, activation='relu', padding='same'), # 28x28x128
            layers.Conv2DTranspose(64, (3,3), strides=2, activation='relu', padding='same'),  # 56x56x64
            layers.Conv2DTranspose(32, (3,3), strides=2, activation='relu', padding='same'),  # 112x112x32
            layers.Conv2DTranspose(3, (3,3), strides=2, activation='sigmoid', padding='same') # 224x224x3
        ])

    def call(self, input_data):
        encoded = self.encoder(input_data)
        decoded = self.decoder(encoded)
        return decoded


In [14]:
import tensorflow as tf
import tensorflow.image as tfi

def psnr_metric(y_true, y_pred):
    return tfi.psnr(y_true, y_pred, max_val=1.0)

def ssim_metric(y_true, y_pred):
    return tfi.ssim(y_true, y_pred, max_val=1.0)


In [18]:

#compile
autoencoder=ConvAutoencoder()
autoencoder.compile(
    optimizer='adam',
    loss='mse',
    metrics=[psnr_metric, ssim_metric]
)


In [19]:
history = autoencoder.fit(
    x=noisy,
    y=clean,
    epochs=20,
    batch_size=32,
    validation_data=(noisy_val_images, clean_val_images)
)


NameError: name 'noisy_train_images' is not defined